# RNN

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas_datareader as pdr
from sklearn import metrics


In [15]:
#Obtaining the Dataset
df = pdr.get_data_yahoo("AAPL", "2010-11-01", "2020-11-01") 


In [16]:
df.head()


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-11-01,10.914286,10.792857,10.793571,10.863571,423889200.0,9.357483
2010-11-02,11.078214,10.964286,10.964286,11.048571,433930000.0,9.516835
2010-11-03,11.174286,11.018929,11.120357,11.171429,508348400.0,9.622657
2010-11-04,11.435000,11.251071,11.266071,11.366786,642488000.0,9.790934
2010-11-05,11.413214,11.312500,11.356786,11.326071,361253200.0,9.755861


In [17]:
df["Diff"] = df.Close.diff()
df["SMA_2"] = df.Close.rolling(2).mean()# Calculating moving averages of closing prices
df["Force_Index"] = df["Close"] * df["Volume"]
df["y"] = df["Diff"].apply(lambda x: 1 if x > 0 else 0).shift(-1)
df = df.drop(
 ["Open", "High", "Low", "Close", "Volume", "Diff", "Adj Close"], # Open,High,Low,Close,Volume,Diff,Adj Close features whose tuples that have missing values are removed fr
 axis=1,
).dropna()
# print(df)
X = df.drop(["y"], axis=1).values # Assigning the independent variable or X variable
y = df["y"].values #Assigning predictor variables


In [18]:
#Splitting into train (80% of the dataset)and test data (20% of the dataset)
X_train, X_test, y_train, y_test = train_test_split(
 X,
 y,
 test_size=0.2,
 shuffle=False,
)


In [19]:
df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2516 entries, 2010-11-02 to 2020-10-29
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   SMA_2        2516 non-null   float64
 1   Force_Index  2516 non-null   float64
 2   y            2516 non-null   float64
dtypes: float64(3)
memory usage: 78.6 KB


In [20]:
df.describe()


,SMA_2,Force_Index,y
count,2516.000000,2.516000e+03,2516.00000
mean,34.327839,6.927472e+09,0.52663
std,21.842704,4.443326e+09,0.49939
min,10.751607,1.282891e+09,0.00000
25%,19.155267,4.111956e+09,0.00000
50%,27.793750,5.590313e+09,1.00000
75%,43.315938,8.246173e+09,1.00000
max,132.789993,4.353884e+10,1.00000


In [21]:
from keras import Sequential
from keras.layers import Dense, SimpleRNN

In [23]:
#Creating the model havng a simple RNN layer and densenet layer with sigmoid activation and adam optimizer
model = Sequential()
model.add(SimpleRNN(2, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 2)                 8         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 3         
Total params: 11
Trainable params: 11
Non-trainable params: 0
_________________________________________________________________


In [24]:
#Training the model

model.fit(X_train[:, :, np.newaxis], y_train, epochs=100)

Epoch 1/100
63/63 [==============================] - 1s 4ms/step - loss: 0.7052 - acc: 0.4661
Epoch 2/100
63/63 [==============================] - 0s 3ms/step - loss: 0.6977 - acc: 0.4731
Epoch 3/100
63/63 [==============================] - 0s 4ms/step - loss: 0.6944 - acc: 0.4733
Epoch 4/100
63/63 [==============================] - 0s 4ms/step - loss: 0.6929 - acc: 0.5317
Epoch 5/100
63/63 [==============================] - 0s 4ms/step - loss: 0.6929 - acc: 0.5106
Epoch 6/100
63/63 [==============================] - 0s 4ms/step - loss: 0.6924 - acc: 0.5202
Epoch 7/100
63/63 [==============================] - 0s 4ms/step - loss: 0.6939 - acc: 0.4972
Epoch 8/100
63/63 [==============================] - 0s 3ms/step - loss: 0.6920 - acc: 0.5258
Epoch 9/100
63/63 [==============================] - 0s 4ms/step - loss: 0.6926 - acc: 0.5179
Epoch 10/100
63/63 [==============================] - 0s 4ms/step - loss: 0.6918 - acc: 0.5277
Epoch 11/100
63/63 [==============================] - 0s 4m

In [26]:
#Testing the model

from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test[:, :, np.newaxis])
print("Accuracy :",accuracy_score(y_test, y_pred > 0.5))

Accuracy : 0.5496031746031746


In [27]:
#Actual and predicted values of the dependent variable

df2 = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted' : y_pred.flatten()})
df2.head()

,Actual,Predicted
0,1.0,0.520369
1,1.0,0.520369
2,0.0,0.520369
3,0.0,0.520369
4,1.0,0.520369


In [28]:
import math
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test,y_pred)*100)
print('Mean Squared Error:', metrics.mean_squared_error(y_test,y_pred))
print('Root Mean Squared Error:', math.sqrt(metrics.mean_squared_error(y_test,y_pred)))

Mean Absolute Error: 49.79792428868158
Mean Squared Error: 0.24839414848690058
Root Mean Squared Error: 0.4983915614122099


# LSTM

In [7]:
#Import the libraries
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [2]:
#Create a 2-D feature NumPy array with random integers
features = (np.random.randint(10, size=(100, 1)))
print(features.shape)

(100, 1)


In [3]:
#Split the dataset into 75/25 for train and test.

training_dataset_length = math.ceil(len(features) * .75)
print(training_dataset_length)

75


In [4]:
#Preprocess the data
#Scale the all of the data to be values between 0 and 1 
scaler = MinMaxScaler(feature_range=(0, 1)) 
scaled_data = scaler.fit_transform(features)

In [5]:
train_data = scaled_data[0:training_dataset_length  , : ]

#Splitting the data
x_train=[]
y_train = []

for i in range(10, len(train_data)):
    x_train.append(train_data[i-10:i,0])
    y_train.append(train_data[i,0])

In [6]:
#Convert to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

#Reshape the data into 3-D array, shape accepted by the LSTM model.
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

In [8]:
from keras.layers import Dropout

# Initialising the RNN
model = Sequential()

model.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(Dropout(0.2))

# Adding a second LSTM layer and Dropout layer
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

# Adding a third LSTM layer and Dropout layer
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

# Adding a fourth LSTM layer and and Dropout layer
model.add(LSTM(units = 50))
model.add(Dropout(0.2))

# Adding the output layer
# For Full connection layer we use dense
# As the output is 1D so we use unit=1
model.add(Dense(units = 1))

In [10]:
#compile and fit the model on 30 epochs
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(x_train, y_train, epochs = 30, batch_size = 50)

Epoch 1/30
2/2 [==============================] - 5s 12ms/step - loss: 0.1214
Epoch 2/30
2/2 [==============================] - 0s 10ms/step - loss: 0.1090
Epoch 3/30
2/2 [==============================] - 0s 9ms/step - loss: 0.1122
Epoch 4/30
2/2 [==============================] - 0s 10ms/step - loss: 0.1147
Epoch 5/30
2/2 [==============================] - 0s 10ms/step - loss: 0.1101
Epoch 6/30
2/2 [==============================] - 0s 10ms/step - loss: 0.1191
Epoch 7/30
2/2 [==============================] - 0s 9ms/step - loss: 0.1098
Epoch 8/30
2/2 [==============================] - 0s 11ms/step - loss: 0.1070
Epoch 9/30
2/2 [==============================] - 0s 10ms/step - loss: 0.1139
Epoch 10/30
2/2 [==============================] - 0s 10ms/step - loss: 0.1119
Epoch 11/30
2/2 [==============================] - 0s 9ms/step - loss: 0.1059
Epoch 12/30
2/2 [==============================] - 0s 10ms/step - loss: 0.1147
Epoch 13/30
2/2 [==============================] - 0s 10ms/step 

In [11]:
#Test data set
test_data = scaled_data[training_dataset_length - 10: , : ]

#splitting the x_test and y_test data sets
x_test = []
y_test =  features[training_dataset_length : , : ] 

for i in range(10,len(test_data)):
    x_test.append(test_data[i-10:i,0])
    
#Convert x_test to a numpy array 
x_test = np.array(x_test)

#Reshape the data into 3-D array
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))

In [12]:
#check predicted values
predictions = model.predict(x_test) 
#Undo scaling
predictions = scaler.inverse_transform(predictions)

#Calculate RMSE score
rmse=np.sqrt(np.mean(((predictions- y_test)**2)))
rmse

3.187683849948664